In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# # 모델 로드
# age_model = load_model("C:/Users/KimDongyoung/Desktop/이미지/age_model.keras")
# gender_model = load_model("C:/Users/KimDongyoung/Desktop/이미지/gender_model.keras")

# # Haar Cascade 얼굴 인식 모델 로드
# face_cascade = cv2.CascadeClassifier("C:/Users/KimDongyoung/Desktop/이미지/haarcascade_frontalface_default.xml")

# # 웹캠 불러오기
# camera = cv2.VideoCapture(0)

# # 웹캠 해상도 설정
# camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 해상도를 1280x720으로 변경
# camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# image_size = 64  # 모델에 맞는 이미지 크기

# while True:
#     ret, frame = camera.read()
#     if not ret:  # 웹캠에서 프레임을 읽지 못한 경우
#         print("웹캠에서 프레임을 읽을 수 없습니다.")
#         break

#     # 얼굴 탐지
#     faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=8)

#     for (x, y, w, h) in faces:
#         img = frame[y:y + h, x:x + w]
#         img = cv2.resize(img, (image_size, image_size))

#         # 예측 수행
#         age_predict = age_model.predict(np.array(img).reshape(-1, image_size, image_size, 3))
#         gender_predict = gender_model.predict(np.array(img).reshape(-1, image_size, image_size, 3))

#         gend = np.round(gender_predict)
#         col = (255, 255, 0) if gend == 0 else (203, 12, 255)
#         gend_text = 'Man' if gend == 0 else 'Woman'

#         # 결과 표시
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 225, 0), 1)
#         cv2.putText(frame, f"Age: {int(np.round(age_predict))} / {gend_text}",
#                     (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, w * 0.005, col, 1)

#     cv2.imshow("camera", frame)

#     key = cv2.waitKey(1)  # 키 입력 대기
#     if key == 27 or key == ord('q'):  # ESC 또는 'q' 키로 종료
#         break

# # 자원 해제
# camera.release()          # 웹캠 자원 해제
# cv2.destroyAllWindows()   # 모든 OpenCV 창 닫기

In [3]:
# Haarcascade 파일 경로 설정
face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Haarcascade 파일 로드
face_cascade = cv2.CascadeClassifier(face_cascade_path)
if face_cascade.empty():
    raise IOError('Haarcascade 파일을 로드할 수 없습니다.')

# 웹캠 초기화
camera = cv2.VideoCapture(0)

# 이미지 크기 설정
image_size = 224

# 모델 로드
age_model = load_model("C:/Users/KimDongyoung/Desktop/이미지/age_model.keras")
gender_model = load_model("C:/Users/KimDongyoung/Desktop/이미지/gender_model.keras")

while True:
    ret, frame = camera.read()
    if not ret:  # 웹캠에서 프레임을 읽지 못한 경우
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # 얼굴 탐지
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=8)

    age_ = []
    gender_ = []
    for (x, y, w, h) in faces:
        img = frame[y:y + h, x:x + w]
        img = cv2.resize(img, (image_size, image_size))
        img = np.array(img).reshape(-1, image_size, image_size, 3) / 255.0  # 입력 데이터 크기 조정 및 정규화
        img = img.astype('float32')  # 데이터 타입을 float32로 변환

        # 모델의 입력 크기 확인
        model_input_shape = age_model.input_shape[1:]  # (224, 224, 3)과 같은 형태

        # 입력 데이터 크기 조정
        if img.shape[1:] != model_input_shape:
            img = cv2.resize(img[0], (model_input_shape[0], model_input_shape[1]))  # img[0]을 사용하여 크기 조정
            img = np.array(img).reshape(-1, model_input_shape[0], model_input_shape[1], model_input_shape[2]) / 255.0
            img = img.astype('float32')

        age_predict = age_model.predict(img)
        gender_predict = gender_model.predict(img)
        age_.append(age_predict)
        gender_.append(np.round(gender_predict))
        gend = np.round(gender_predict)
        if gend == 0:
            gend = 'Man'
            col = (255, 255, 0)
        else:
            gend = 'Woman'
            col = (203, 12, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 225, 0), 1)
        cv2.putText(frame, "Age:" + str(int(np.round(age_predict))) + " / " + str(gend),
                    (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, w * 0.005, col, 1)
    cv2.imshow("camera", frame)

    key = cv2.waitKey(10)
    # 자판의 ESC키가 27, ESC 입력시 비디오창 종료
    if key == 27:
        cv2.destroyAllqWindows()
        break

# 자원 해제
camera.release()          # 웹캠 자원 해제
cv2.destroyAllWindows()   # 모든 OpenCV 창 닫기

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_19532\621038514.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, "Age:" + str(int(np.round(age_predict))) + " / " + str(gend),


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━

AttributeError: module 'cv2' has no attribute 'destroyAllqWindows'

: 

In [7]:
import os
import pandas as pd

In [15]:
# Haarcascade 파일 경로 설정
face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Haarcascade 파일 로드
face_cascade = cv2.CascadeClassifier(face_cascade_path)
if face_cascade.empty():
    raise IOError('Haarcascade 파일을 로드할 수 없습니다.')

# 웹캠 초기화
camera = cv2.VideoCapture(0)

# 이미지 크기 설정
image_size = 224

# 모델 로드
age_model = load_model("C:/Users/KimDongyoung/Desktop/이미지/age_model.keras")
gender_model = load_model("C:/Users/KimDongyoung/Desktop/이미지/gender_model.keras")

# 결과를 저장할 리스트 초기화
results = []

# 이미지 저장 
# 폴더 생성
if not os.path.exists('images'):
    os.makedirs('images')

while True:
    ret, frame = camera.read()
    if not ret:  # 웹캠에서 프레임을 읽지 못한 경우
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # 얼굴 탐지
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=8)

    for (x, y, w, h) in faces:
        img = frame[y:y + h, x:x + w]
        img = cv2.resize(img, (image_size, image_size))
        img = np.array(img).reshape(-1, image_size, image_size, 3) / 255.0  # 입력 데이터 크기 조정 및 정규화
        img = img.astype('float32')  # 데이터 타입을 float32로 변환

        # 모델의 입력 크기 확인
        model_input_shape = age_model.input_shape[1:]  # (224, 224, 3)과 같은 형태

        # 입력 데이터 크기 조정
        if img.shape[1:] != model_input_shape:
            img = cv2.resize(img[0], (model_input_shape[0], model_input_shape[1]))  # img[0]을 사용하여 크기 조정
            img = np.array(img).reshape(-1, model_input_shape[0], model_input_shape[1], model_input_shape[2]) / 255.0
            img = img.astype('float32')

        age_predict = age_model.predict(img)
        gender_predict = gender_model.predict(img)
        age_ = int(np.round(age_predict[0][0]))
        gender_ = 'Man' if np.round(gender_predict[0][0]) == 0 else 'Woman'

        # 결과를 리스트에 저장
        results.append({
            'image_name': f'image_{len(results) + 1}.jpg',
            'age': age_,
            'gender': gender_
        })

        # 얼굴 부분만 저장
        face_img = frame[y:y + h, x:x + w]
        cv2.imwrite(os.path.join('images', f'image_{len(results)}.jpg'), face_img)

        # 결과 표시
        col = (255, 255, 0) if gender_ == 'Man' else (203, 12, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 225, 0), 1)
        cv2.putText(frame, f"Age: {age_} / {gender_}",
                    (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, w * 0.005, col, 1)

    cv2.imshow("camera", frame)

    key = cv2.waitKey(10)
    # 자판의 ESC키가 27, ESC 입력시 비디오창 종료
    if key == 27:
        cv2.destroyAllWindows()
        break

# 자원 해제
camera.release()          # 웹캠 자원 해제
cv2.destroyAllWindows()   # 모든 OpenCV 창 닫기

# 결과를 CSV 파일로 저장
df_results = pd.DataFrame(results)
df_results.to_csv('results.csv', index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━

In [11]:
def delete_all_but_first_image(folder_path):
    """
    'images' 폴더에 저장된 모든 파일 중 맨 처음에 찍힌 사진을 제외한 나머지 파일을 삭제합니다.
    """
    # 폴더 내 모든 파일 목록을 가져옵니다.
    files = sorted(os.listdir(folder_path))
    
    # 맨 처음에 찍힌 사진을 제외한 나머지 파일을 삭제합니다.
    if len(files) > 1:
        for file in files[1:]:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
    else:
        print("폴더에 파일이 하나만 있거나 비어 있습니다.")

# 사용 예시
delete_all_but_first_image('images')

Deleted: images\image_10.jpg
Deleted: images\image_11.jpg
Deleted: images\image_12.jpg
Deleted: images\image_13.jpg
Deleted: images\image_14.jpg
Deleted: images\image_15.jpg
Deleted: images\image_16.jpg
Deleted: images\image_17.jpg
Deleted: images\image_18.jpg
Deleted: images\image_19.jpg
Deleted: images\image_2.jpg
Deleted: images\image_20.jpg
Deleted: images\image_21.jpg
Deleted: images\image_22.jpg
Deleted: images\image_23.jpg
Deleted: images\image_24.jpg
Deleted: images\image_25.jpg
Deleted: images\image_26.jpg
Deleted: images\image_27.jpg
Deleted: images\image_28.jpg
Deleted: images\image_29.jpg
Deleted: images\image_3.jpg
Deleted: images\image_30.jpg
Deleted: images\image_31.jpg
Deleted: images\image_32.jpg
Deleted: images\image_33.jpg
Deleted: images\image_34.jpg
Deleted: images\image_35.jpg
Deleted: images\image_36.jpg
Deleted: images\image_37.jpg
Deleted: images\image_4.jpg
Deleted: images\image_5.jpg
Deleted: images\image_6.jpg
Deleted: images\image_7.jpg
Deleted: images\imag

In [12]:
def keep_first_csv_and_delete_others(csv_path):
    """
    CSV 파일 중 맨 처음에 저장된 파일을 제외한 나머지 파일을 삭제합니다.
    """
    # 폴더 내 모든 CSV 파일 목록을 가져옵니다.
    files = sorted([f for f in os.listdir(csv_path) if f.endswith('.csv')])
    
    # 맨 처음에 저장된 CSV 파일을 제외한 나머지 파일을 삭제합니다.
    if len(files) > 1:
        for file in files[1:]:
            file_path = os.path.join(csv_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
    else:
        print("폴더에 CSV 파일이 하나만 있거나 비어 있습니다.")

keep_first_csv_and_delete_others('.')

Deleted: .\1012_roberta_base(batch_64_max64)__.csv
Deleted: .\results.csv
Deleted: .\soft_voting_logNrf_DY.csv
Deleted: .\test_df_1007.csv
Deleted: .\test_df_1008.csv
Deleted: .\train_df_1007.csv
Deleted: .\train_df_1008.csv
